In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine,inspect
from sqlalchemy.orm import Session
import pandas as pd
from flask import Flask, jsonify

In [2]:
engine = create_engine("sqlite:///db/bellybutton.sqlite")
session = Session(engine)
Base = automap_base()
Base.prepare(engine, reflect=True)

In [3]:
Base.classes.keys() # Get the table names
# Base.metadata.tables # Check tables, not much useful

['sample_metadata', 'samples']

In [4]:
Samples_Metadata = Base.classes.sample_metadata
Samples = Base.classes.samples

In [5]:
inspector = inspect(engine)
inspector.get_table_names()

['sample_metadata', 'samples']

In [6]:
columns = inspector.get_columns('sample_metadata')
for column in columns:
    primarykeystr = ""
    if column['primary_key'] == 1:
        primarykeystr = "Primary Key"
    print(column["name"],column["type"],primarykeystr)

sample INTEGER Primary Key
EVENT TEXT 
ETHNICITY TEXT 
GENDER TEXT 
AGE REAL 
WFREQ REAL 
BBTYPE TEXT 
LOCATION TEXT 
COUNTRY012 TEXT 
ZIP012 TEXT 
COUNTRY1319 TEXT 
ZIP1319 TEXT 
DOG TEXT 
CAT TEXT 
IMPSURFACE013 REAL 
NPP013 REAL 
MMAXTEMP013 REAL 
PFC013 REAL 
IMPSURFACE1319 TEXT 
NPP1319 REAL 
MMAXTEMP1319 REAL 
PFC1319 REAL 


In [ ]:
columns = inspector.get_columns('samples')
for column in columns:
    primarykeystr = ""
    if column['primary_key'] == 1:
        primarykeystr = "Primary Key"
    print(column["name"], column["type"], primarykeystr)

In [8]:
stmt = session.query(Samples).statement
df = pd.read_sql_query(stmt, session.bind)
df.head()


,otu_id,otu_label,940,941,943,944,945,946,947,948,...,1562,1563,1564,1572,1573,1574,1576,1577,1581,1601
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...,0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
columnnames = list(df.columns)[2:]
columnnames
# ['940',
#  '941',
#  '943',
#  '944',
#  '945',
#  '946',
#  '947',

In [10]:
# @app.route("/names")
def names():
    """Return a list of sample names."""

    # Use Pandas to perform the sql query
    stmt = session.query(Samples).statement
    df = pd.read_sql_query(stmt, session.bind)
    
    print(list(df.columns)[2:])

    # Return a list of the column names (sample names)
#     return jsonify(list(df.columns)[2:])

In [11]:
names()

['940', '941', '943', '944', '945', '946', '947', '948', '949', '950', '952', '953', '954', '955', '956', '958', '959', '960', '961', '962', '963', '964', '966', '967', '968', '969', '970', '971', '972', '973', '974', '975', '978', '1233', '1234', '1235', '1236', '1237', '1238', '1242', '1243', '1246', '1253', '1254', '1258', '1259', '1260', '1264', '1265', '1273', '1275', '1276', '1277', '1278', '1279', '1280', '1281', '1282', '1283', '1284', '1285', '1286', '1287', '1288', '1289', '1290', '1291', '1292', '1293', '1294', '1295', '1296', '1297', '1298', '1308', '1309', '1310', '1374', '1415', '1439', '1441', '1443', '1486', '1487', '1489', '1490', '1491', '1494', '1495', '1497', '1499', '1500', '1501', '1502', '1503', '1504', '1505', '1506', '1507', '1508', '1510', '1511', '1512', '1513', '1514', '1515', '1516', '1517', '1518', '1519', '1521', '1524', '1526', '1527', '1530', '1531', '1532', '1533', '1534', '1535', '1536', '1537', '1539', '1540', '1541', '1542', '1543', '1544', '1545', 

In [ ]:
sel = [Samples_Metadata.sample]
queryresult = session.query(*sel).all()
queryresult
columnnames = list(df.columns)[2:]
columnnames
# ['940',
#  '941',
#  '943',
#  '944',
#  '945',
#  '946',
#  '947',

In [13]:
# @app.route("/metadata/<sample>")
def sample_metadata(sample):
    """Return the MetaData for a given sample."""
    sel = [
        Samples_Metadata.sample,
        Samples_Metadata.ETHNICITY,
        Samples_Metadata.GENDER,
        Samples_Metadata.AGE,
        Samples_Metadata.LOCATION,
        Samples_Metadata.BBTYPE,
        Samples_Metadata.WFREQ,
    ]

    results = session.query(*sel).filter(Samples_Metadata.sample == sample).all()

    # Create a dictionary entry for each row of metadata information
    sample_metadata = {}
    for result in results:
        sample_metadata["sample"] = result[0]
        sample_metadata["ETHNICITY"] = result[1]
        sample_metadata["GENDER"] = result[2]
        sample_metadata["AGE"] = result[3]
        sample_metadata["LOCATION"] = result[4]
        sample_metadata["BBTYPE"] = result[5]
        sample_metadata["WFREQ"] = result[6]

    print(sample_metadata)
#     return jsonify(sample_metadata)

In [14]:
sample_metadata('940')

{'sample': 940, 'ETHNICITY': 'Caucasian', 'GENDER': 'F', 'AGE': 24.0, 'LOCATION': 'Beaufort/NC', 'BBTYPE': 'I', 'WFREQ': 2.0}


In [15]:
# @app.route("/samples/<sample>")
def samples(sample):
    """Return `otu_ids`, `otu_labels`,and `sample_values`."""
    stmt = session.query(Samples).statement
    df = pd.read_sql_query(stmt, session.bind)
    
    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df.loc[df[sample] > 1, ["otu_id", "otu_label", sample]]

    # Sort by sample
    sample_data.sort_values(by=sample, ascending=False, inplace=True)

    # Format the data to send as json
    data = {
        "otu_ids": sample_data.otu_id.values.tolist(),
        "sample_values": sample_data[sample].values.tolist(),
        "otu_labels": sample_data.otu_label.tolist(),
    }
    
    print(data)
#     return jsonify(data)


In [ ]:
samples('940')
# {'otu_ids': [1167, 2859, 482, 2264, 41, 1189, 352, 18